## Data Analysis using BulkLMM - BXD Longevity Study

In [1]:
using CSV, DelimitedFiles, DataFrames, Missings, XLSX
using LinearAlgebra, Statistics, Optim
using Random, Distributions, LoopVectorization
# using GeneNetworkAPI, Downloads
using BenchmarkTools

In [2]:
using Plots

In [3]:
local_path = "../../BulkLMM.jl/src";

## Load processed_data:

### By individuals:

In [4]:
pwd()

"/home/zyu20/git/BulkLMM_Analyses/BXDLongevity"

In [5]:
BXD_pheno_ind_summary_df = CSV.read("../../../shareddata/BXDLongevity/data/GN886_pheno_summary.csv", DataFrame);

By individuals, there are 248 individual samples for 32445 livear proteome.

In [6]:
println(size(BXD_pheno_ind_summary_df)) 
BXD_pheno_ind_summary_df[1:10, 1:10]

(248, 32448)


Row,Sample,Strain,Strain_num,P42209_DESGLNRK_2,P42209_GLRPLDVAFLR_3,Q99M02_VGDPVYR_2,Q99M02_VWIYPIK_2,Q99M02_LCDPSVK_2,Q99M02_CVLTTVDPDTGIIDR_2,Q99M02_LVQFDTSMK_2
,String7,String7,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,H1009,BXD9,9,11.349,11.534,17.587,17.517,17.309,16.574,16.323
2,H0370,BXD9,9,11.249,12.735,17.427,17.325,17.252,16.685,16.021
3,H2577,BXD9,9,12.415,10.487,17.89,17.488,17.594,16.731,16.208
4,H0365,BXD9,9,11.374,10.674,17.714,17.401,17.381,16.621,16.337
5,H1333,BXD13,13,11.687,11.524,17.362,17.367,17.071,16.465,15.97
6,H2259,BXD24,24,11.837,11.715,17.57,17.792,17.505,16.894,16.277
7,H1792,BXD24,24,11.563,11.434,17.789,17.847,17.416,17.042,16.292
8,H1791,BXD24,24,12.5,12.273,17.944,17.833,17.63,16.862,16.3
9,H1541,BXD24,24,11.815,11.564,17.794,17.759,17.456,16.878,16.377


In [7]:
BXD_geno_ind_summary_df = CSV.read("../../../shareddata/BXDLongevity/data/GN886_geno_summary.csv", DataFrame);

By individuals, there are 248 individual samples for 7321 markers.

In [8]:
println(size(BXD_geno_ind_summary_df)) 
BXD_geno_ind_summary_df[1:10, 1:10]

(248, 7324)


Row,Sample,Strain,Strain_num,rs31443144,rs6269442,rs32285189,rs258367496,rs32430919,rs36251697,rs30658298
,String7,String7,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,H1009,BXD9,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,H0370,BXD9,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,H2577,BXD9,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,H0365,BXD9,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,H1333,BXD13,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,H2259,BXD24,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,H1792,BXD24,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,H1791,BXD24,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,H1541,BXD24,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### By strains:

In [9]:
pwd()

"/home/zyu20/git/BulkLMM_Analyses/BXDLongevity"

In [10]:
BXD_pheno_strains_summary_df = CSV.read("../../../shareddata/BXDLongevity/data/GN886_pheno_strain_means_summary.csv", DataFrame);

By strains, there are 50 BXD strains for 32445 liver proteome.

In [11]:
println(size(BXD_pheno_strains_summary_df)) 
BXD_pheno_strains_summary_df[1:10, 1:10]

(50, 32447)


Row,Strain,Number of Samples,P42209_DESGLNRK_2,P42209_GLRPLDVAFLR_3,Q99M02_VGDPVYR_2,Q99M02_VWIYPIK_2,Q99M02_LCDPSVK_2,Q99M02_CVLTTVDPDTGIIDR_2,Q99M02_LVQFDTSMK_2,Q99M02_QLQQVGTVSK_2
,String7,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,BXD9,4,11.5968,11.3575,17.6545,17.4327,17.384,16.6528,16.2222,16.3227
2,BXD13,1,11.687,11.524,17.362,17.367,17.071,16.465,15.97,15.98
3,BXD24,5,12.0778,11.7458,17.7926,17.8152,17.5168,16.9196,16.3012,16.4304
4,BXD27,3,11.717,11.149,17.336,17.3143,17.165,16.5283,15.9407,16.002
5,BXD29,11,11.735,11.3496,17.4583,17.3877,17.2431,16.4945,16.0869,16.1075
6,BXD32,8,11.6266,11.554,17.8254,17.7716,17.4639,16.9029,16.434,16.425
7,BXD34,13,11.7491,11.9135,17.6108,17.5974,17.2988,16.7761,16.3147,16.2926
8,BXD39,5,11.9158,11.719,17.395,17.385,17.157,16.5226,16.179,16.0594
9,BXD40,8,11.8651,11.6817,17.5986,17.6386,17.3861,16.6824,16.2855,16.2523


In [12]:
BXD_geno_strains_summary_df = CSV.read("../../../shareddata/BXDLongevity/data/GN886_geno_strains_summary.csv", DataFrame);

By strains, there are 50 BXD strains for 7321 markers.

In [13]:
println(size(BXD_geno_strains_summary_df)) 
BXD_geno_strains_summary_df[1:10, 1:10]

(50, 7322)


Row,Strain,rs31443144,rs6269442,rs32285189,rs258367496,rs32430919,rs36251697,rs30658298,rs51852623,rs31879829
,String7,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,BXD9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BXD13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BXD24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BXD27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,BXD29,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,BXD32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,BXD34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,BXD39,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,BXD40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Load functions:

In [14]:
pwd()

"/home/zyu20/git/BulkLMM_Analyses/BXDLongevity"

In [71]:
include(joinpath(local_path, "kinship.jl"));
include(joinpath(local_path, "util.jl"));
include(joinpath(local_path, "wls.jl"));
include(joinpath(local_path, "lmm.jl"));
include(joinpath(local_path, "gridbrent.jl"));
include(joinpath(local_path, "transform_helpers.jl"));
include(joinpath(local_path, "scan.jl"));
include(joinpath(local_path, "bulkscan_helpers.jl"));
include(joinpath(local_path, "bulkscan.jl"));
include(joinpath(local_path, "readData.jl"));
include(joinpath(local_path, "../plot_utils/visuals_utils.jl"));
include(joinpath(local_path, "analysis_helpers/single_trait_analysis.jl"));
include("../../BigRiver_util_code/src/kinship_utils.jl");

## Objectives

- gemma one trait, ind and strains

- scan_alt one trait, ind and strains

- scan_null one trait, ind and strains

- bulkscan_grid (null), all traits, ...

- data structure for all traits results...

- Heatmap grids_taken, compare on strain means and individuals

- Histogram of heritabilities (null) for all traits (both strain means and individual level data)

- Loglikelihood by h2_grid plot, strain means v.s. individuals

- Report fixed effects:

- Adjusting difference in the number of samples across strains

- Histogram of heritabilities (null) for all traits (both strain means and individual level data)

## Run BulkLMM.jl:

### Calculate kinship:

First, calculate the kinship matrix of relatedness among the strains (50-by-50):

In [18]:
geno_strains = Matrix{Float64}(BXD_geno_strains_summary_df[:, 2:end]);

In [19]:
kinship_strains = calcKinship(geno_strains);

Then, construct the kinship matrix of relatedness among the individuals based on which strain they are from:

In [20]:
BXD_geno_ind_summary_df.Strain_num = Vector{Int64}(BXD_geno_ind_summary_df.Strain_num);

In [21]:
@time kinship_ind_from_strains = calcKinship2(kinship_strains, BXD_geno_ind_summary_df.Strain_num);

  0.082343 seconds (103.47 k allocations: 5.996 MiB, 96.17% compilation time)


### Single-trait genome scans:

To see single trait genome scan results, we take the 29437-th livear proteomic as the single trait to run analysis:  

In [22]:
lp_names = names(BXD_pheno_ind_summary_df)[4:end];

In [23]:
lp_id = 29437;
lp_names[lp_id]

"Q9Z2I8_SSGLPITSAVDLEDAAK_3"

In [24]:
prior = [1.0, 0.0];

#### For individual liver proteome

In [25]:
geno_ind = Matrix{Float64}(BXD_geno_ind_summary_df[:, 4:end]);

In [26]:
lp_ind_Y = Matrix{Float64}(BXD_pheno_ind_summary_df[:, 4:end]);
lp_ind_y = reshape(lp_ind_Y[:, lp_id], :, 1);

In [27]:
lpst_ind_Y = colStandardize(lp_ind_Y);
lpst_ind_y = reshape(lpst_ind_Y[:, lp_id], :, 1);

In [28]:
@time scan_results_ind = scan(lpst_ind_y, geno_ind, kinship_ind_from_strains;
                              prior_variance = 1.0, prior_sample_size = 0.1);

  0.677914 seconds (735.36 k allocations: 175.558 MiB, 9.97% gc time, 68.97% compilation time)


In [29]:
@time scan_results_ind_alt = scan(lpst_ind_y, geno_ind, kinship_ind_from_strains;
                                  assumption = "alt", 
                                  prior_variance = prior[1], prior_sample_size = prior[2]);

 13.563152 seconds (12.12 M allocations: 14.708 GiB, 7.66% gc time, 0.27% compilation time)


In [30]:
@time scan_perms_results_ind = scan(lpst_ind_y, geno_ind, kinship_ind_from_strains;
                                    prior_variance = prior[1], prior_sample_size = prior[2],
                                    permutation_test = true, nperms = 1000, original = true);

  0.988747 seconds (2.06 M allocations: 269.181 MiB, 2.34% gc time, 88.05% compilation time)


In [31]:
single_results_ind = DataFrame(hcat(scan_results_ind.lod, scan_results_ind_alt.lod, scan_perms_results_ind[:, 1]), ["scan_null", "scan_alt", "scan_perms.original"]);

In [32]:
single_results_ind[1:6, :]

Row,scan_null,scan_alt,scan_perms.original
,Float64,Float64,Float64
1,0.44861,0.44861,0.44861
2,0.44861,0.44861,0.44861
3,0.44861,0.44861,0.44861
4,0.44861,0.44861,0.44861
5,0.44861,0.44861,0.44861
6,0.44861,0.44861,0.44861


#### For liver proteome means by strains

In [33]:
lp_strains_Y = Matrix{Float64}(BXD_pheno_strains_summary_df[:, 3:end]);
lp_strains_y = reshape(lp_strains_Y[:, lp_id], :, 1);

In [34]:
lpst_strains_Y = colStandardize(lp_strains_Y);
lpst_strains_y = reshape(lpst_strains_Y[:, lp_id], :, 1);

In [35]:
@time scan_results_strains = scan(lpst_strains_y, geno_strains, kinship_strains; 
                                  prior_variance = prior[1], prior_sample_size = prior[2],
                                  optim_interval = 4);

  0.081825 seconds (206.70 k allocations: 40.364 MiB, 42.62% compilation time)


In [36]:
@time scan_results_strains_alt = scan(lpst_strains_y, geno_strains, kinship_strains;
                                      assumption = "alt",
                                      prior_variance = prior[1], prior_sample_size = prior[2]);

  3.073053 seconds (7.03 M allocations: 2.140 GiB, 19.08% gc time)


In [37]:
@time scan_perms_results_strains = scan(lpst_strains_y, geno_strains, kinship_strains;
                                        permutation_test = true, nperms = 1000, original = true, 
                                        prior_variance = prior[1], prior_sample_size = prior[2]);

  0.079482 seconds (215.13 k allocations: 86.077 MiB, 43.30% compilation time)


In [38]:
single_results_strains = DataFrame(hcat(scan_results_strains.lod, scan_results_strains_alt.lod, scan_perms_results_strains[:, 1]), ["scan_null", "scan_alt", "scan_perms.original"]);

In [39]:
single_results_strains[1:6, :]

Row,scan_null,scan_alt,scan_perms.original
,Float64,Float64,Float64
1,0.387771,0.267201,0.387771
2,0.387771,0.267201,0.387771
3,0.387771,0.267201,0.387771
4,0.387771,0.267201,0.387771
5,0.387771,0.267201,0.387771
6,0.387771,0.267201,0.387771


### Multiple-trait genome scans:

In [40]:
BLAS.set_num_threads(Threads.nthreads())

In [41]:
BLAS.get_num_threads()

16

In [42]:
grid_list = collect(0.0:0.05:0.95);
# grid_list = collect(0.0:0.01:0.99);

#### For individual liver proteome

In [43]:
size(lpst_ind_Y)

(248, 32445)

In [44]:
size(lpst_strains_Y)

(50, 32445)

In [45]:
Threads.nthreads()

16

In [46]:
@time bulkscan_results_ind_null = bulkscan_null(lp_ind_Y, geno_ind, kinship_ind_from_strains; nb = Threads.nthreads(),
                                                prior_variance = 1.0, prior_sample_size = 0.0, optim_interval = 1);

169.808281 seconds (2.60 G allocations: 1.813 TiB, 13.55% gc time, 0.20% compilation time)


In [47]:
BLAS.set_num_threads(Threads.nthreads())

In [48]:
@time bulkscan_results_ind_alt_grid = bulkscan_alt_grid(lpst_ind_Y, geno_ind, kinship_ind_from_strains, grid_list);

 30.297903 seconds (8.73 M allocations: 48.920 GiB, 7.60% gc time, 0.14% compilation time)


In [49]:
@time bulkscan_results_ind_null_grid = bulkscan_null_grid(lpst_ind_Y, geno_ind, kinship_ind_from_strains, grid_list);

 11.908529 seconds (13.13 M allocations: 11.768 GiB, 4.30% gc time, 13.09% compilation time)


#### For liver proteome means by strains

In [50]:
@time bulkscan_results_strains_null = bulkscan_null(lpst_strains_Y, geno_strains, kinship_strains; nb = Threads.nthreads(),
                                                    prior_variance = 1.0, prior_sample_size = 0.0, optim_interval = 1);

 67.916028 seconds (2.61 G allocations: 434.477 GiB, 38.85% gc time)


In [51]:
BLAS.set_num_threads(Threads.nthreads())

In [52]:
@time bulkscan_results_strains_alt_grid = bulkscan_alt_grid(lpst_strains_Y, geno_strains, kinship_strains, grid_list);

 22.761734 seconds (8.69 M allocations: 43.938 GiB, 8.99% gc time)


In [53]:
@time bulkscan_results_strains_null_grid = bulkscan_null_grid(lpst_strains_Y, geno_strains, kinship_strains, grid_list;
                                                              prior_variance = 1.0, prior_sample_size = 0.0);

  5.316484 seconds (9.47 M allocations: 5.288 GiB, 12.89% gc time)


In [54]:
pwd()

"/home/zyu20/git/BulkLMM_Analyses/BXDLongevity"

### Peformance check:

#### Individual data:
- n = 248
- m = 32445
- p = 7321

In [40]:
size(lpst_ind_Y)

(248, 32445)

In [41]:
size(geno_ind)

(248, 7321)

In [42]:
@benchmark scan(lpst_ind_y, geno_ind, kinship_ind_from_strains)

BenchmarkTools.Trial: 31 samples with 1 evaluation.
 Range (min … max):  137.718 ms … 295.167 ms  ┊ GC (min … max): 0.00% … 12.62%
 Time  (median):     153.051 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   165.246 ms ±  34.361 ms  ┊ GC (mean ± σ):  8.15% ±  9.55%

  █    ▁                                                         
  █▄▄▁▆█▄▄▁▆▄▄▄▁▄▁▄▁▁▁▄▁▁▁▄▁▄▁▁▁▄▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  138 ms           Histogram: frequency by time          295 ms <

 Memory estimate: 136.22 MiB, allocs estimate: 82175.

In [43]:
@benchmark scan(lpst_ind_y, geno_ind, kinship_ind_from_strains;
                prior_variance = 1.0, prior_sample_size = 0.1)

BenchmarkTools.Trial: 34 samples with 1 evaluation.
 Range (min … max):  137.078 ms … 185.899 ms  ┊ GC (min … max): 0.00% … 9.05%
 Time  (median):     151.067 ms               ┊ GC (median):    4.71%
 Time  (mean ± σ):   147.267 ms ±   9.947 ms  ┊ GC (mean ± σ):  5.02% ± 4.88%

  ▄▃                █                                            
  ██▇▁▁▄▁▁▁▁▁▁▁▁▁▁▄▇█▇▄▁▄▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  137 ms           Histogram: frequency by time          186 ms <

 Memory estimate: 136.25 MiB, allocs estimate: 82196.

In [44]:
@benchmark scan(lpst_ind_y, geno_ind, kinship_ind_from_strains;
                assumption = "alt")

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 13.118 s (6.90% GC) to evaluate,
 with a memory estimate of 14.36 GiB, over 11700339 allocations.

In [45]:
@benchmark scan(lpst_ind_y, geno_ind, kinship_ind_from_strains;
                assumption = "alt", 
                prior_variance = 1.0, prior_sample_size = 0.1)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 13.560 s (6.49% GC) to evaluate,
 with a memory estimate of 14.36 GiB, over 11699184 allocations.

In [46]:
BLAS.get_num_threads()

16

In [69]:
@time bulkscan_null(lpst_ind_Y, geno_ind, kinship_ind_from_strains,
                    prior_variance = 1.0, prior_sample_size = 0.1);

158.591825 seconds (2.60 G allocations: 1.813 TiB, 18.45% gc time)


In [80]:
grid_perf = collect(0.0:0.05:0.95);

#### Strain mean data:
- n = 50
- m = 32445
- p = 7321

In [51]:
size(lpst_strains_Y)

(50, 32445)

In [50]:
size(geno_strains)

(50, 7321)

In [52]:
@benchmark scan(lpst_strains_y, geno_strains, kinship_strains)

BenchmarkTools.Trial: 95 samples with 1 evaluation.
 Range (min … max):  39.550 ms … 241.791 ms  ┊ GC (min … max): 0.00% … 79.77%
 Time  (median):     48.669 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.642 ms ±  27.347 ms  ┊ GC (mean ± σ):  7.67% ± 11.64%

    █▅                                                          
  ▄▁██▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  39.5 ms       Histogram: log(frequency) by time       234 ms <

 Memory estimate: 32.42 MiB, allocs estimate: 81398.

In [53]:
@benchmark scan(lpst_strains_y, geno_strains, kinship_strains;
                prior_variance = 1.0, prior_sample_size = 0.1)

BenchmarkTools.Trial: 94 samples with 1 evaluation.
 Range (min … max):  45.709 ms … 229.799 ms  ┊ GC (min … max): 0.00% … 83.15%
 Time  (median):     49.179 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   53.652 ms ±  25.903 ms  ┊ GC (mean ± σ):  7.42% ± 11.93%

  █▇                                                            
  ███▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▁
  45.7 ms         Histogram: frequency by time          225 ms <

 Memory estimate: 32.32 MiB, allocs estimate: 81020.

In [54]:
@benchmark scan(lpst_strains_y, geno_strains, kinship_strains;
                assumption = "alt")

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.678 s …   2.808 s  ┊ GC (min … max): 5.15% … 9.90%
 Time  (median):     2.743 s              ┊ GC (median):    7.58%
 Time  (mean ± σ):   2.743 s ± 91.680 ms  ┊ GC (mean ± σ):  7.58% ± 3.36%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.68 s         Histogram: frequency by time        2.81 s <

 Memory estimate: 2.05 GiB, allocs estimate: 6725943.

In [55]:
@benchmark scan(lpst_strains_y, geno_strains, kinship_strains;
                assumption = "alt", 
                prior_variance = 1.0, prior_sample_size = 0.1)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.667 s …  1.675 s  ┊ GC (min … max): 8.24% … 8.60%
 Time  (median):     1.668 s             ┊ GC (median):    8.29%
 Time  (mean ± σ):   1.670 s ± 4.477 ms  ┊ GC (mean ± σ):  8.38% ± 0.20%

  █       █                                              █  
  █▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.67 s        Histogram: frequency by time        1.67 s <

 Memory estimate: 1.23 GiB, allocs estimate: 4082862.

In [65]:
BLAS.get_num_threads()

1

In [68]:
@time bulkscan_null(lpst_strains_Y, geno_strains, kinship_strains,
                    prior_variance = 1.0, prior_sample_size = 0.1);

 59.414409 seconds (2.61 G allocations: 434.448 GiB, 40.69% gc time)


In [56]:
grid_perf = collect(0.0:0.05:0.95);

In [57]:
@benchmark bulkscan_null_grid(lpst_strains_Y, geno_strains, kinship_strains, grid_perf;
                              prior_variance = 1.0, prior_sample_size = 0.1)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.319 s …   4.427 s  ┊ GC (min … max): 3.48% … 3.22%
 Time  (median):     4.373 s              ┊ GC (median):    3.35%
 Time  (mean ± σ):   4.373 s ± 75.882 ms  ┊ GC (mean ± σ):  3.35% ± 0.18%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.32 s         Histogram: frequency by time        4.43 s <

 Memory estimate: 5.29 GiB, allocs estimate: 9473686.

In [58]:
@benchmark bulkscan_alt_grid(lpst_strains_Y, geno_strains, kinship_strains, grid_perf)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 14.257 s (4.92% GC) to evaluate,
 with a memory estimate of 36.86 GiB, over 8691487 allocations.

In [42]:
grid_fine = collect(0.00:0.01:0.99);

## Compare with GEMMA:

In [95]:
include("../../BigRiver_util_code/src/run_gemma_utils.jl");

In [96]:
m = size(lpst_ind_Y, 2)

32445

In [97]:
p = size(geno_ind, 2)

7321

In [98]:
samples = sample(1:m, 1000);

In [99]:
marker_names = names(BXD_geno_strains_summary_df)[2:end];

#### strains:

In [100]:
pheno_strains_subset = lpst_strains_Y[:, samples];

In [101]:
t_strains = @elapsed gemma_L_strains = run_gemma(pheno_strains_subset, geno_strains, kinship_strains,
                                                ["A", "B"], marker_names, 
                                                "data/GEMMA_data/GN886_pheno_strains_y.txt", "data/GEMMA_data/GN886_geno_strains.txt", "data/GEMMA_data/kinship_strains.txt", "results_strains",
                                                "/home/zyu20/Softwares/gemma-0.98.5-linux-static-AMD64")

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 50
## number of analyzed individuals = 50
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Start Eigen-Decomposition...
pve estimate =0.72234
se(pve) =0.608306
================================================== 100%


**** INFO: Done.


GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 50
## number of analyzed individuals = 50
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Start Eigen-Decomposition...
pve estimate =0.0732021
se(pve) =0.454556
================================================== 100%


**** INFO: Done.


GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 50
## number of analyzed individuals = 50
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Start Eigen-Decomposition...
pve estimate =0.155141
se(pve) =0.313202
================================================== 100%


...

**** INFO: Done.


GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 50
## number of analyzed individuals = 50
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Start Eigen-Decomposition...
pve estimate =4.75725e-06
se(pve) =0.52578
================================================== 100%


**** INFO: Done.
Excessive output truncated after 524412 bytes.

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 


668.049334446

In [104]:
t_strains

668.049334446

In [106]:
findall(isnan.(gemma_L_strains))

CartesianIndex{2}[]

In [112]:
Threads.nthreads()

16

In [119]:
function run_bulklmm_alt(pheno::Array{Float64, 2}, geno::Array{Float64, 2}, kinship::Array{Float64, 2})
    
    m = size(pheno, 2);
    p = size(geno, 2);
    
    L = zeros(p, m);
    
    Threads.@threads for t in 1:1000
        pheno_y = reshape(pheno[:, t], :, 1);
        L[:, t] .= scan(pheno_y, geno, kinship; assumption = "alt", reml = true).lod;
    end
    
    return L
    
end

run_bulklmm_alt (generic function with 1 method)

In [121]:
BLAS.get_num_threads()

16

In [123]:
BLAS.set_num_threads(4)

In [124]:
@time bulklmm_L_exact_alt = run_bulklmm_alt(pheno_strains_subset, geno_strains, kinship_strains);

LoadError: TaskFailedException

[91m    nested task error: [39mInterruptException:
    Stacktrace:
      [1] [0m[1mArray[22m
    [90m    @ [39m[90m./[39m[90m[4mboot.jl:459[24m[39m[90m [inlined][39m
      [2] [0m[1mArray[22m
    [90m    @ [39m[90m./[39m[90m[4mboot.jl:467[24m[39m[90m [inlined][39m
      [3] [0m[1mzeros[22m
    [90m    @ [39m[90m./[39m[90m[4marray.jl:525[24m[39m[90m [inlined][39m
      [4] [0m[1mzeros[22m
    [90m    @ [39m[90m./[39m[90m[4marray.jl:521[24m[39m[90m [inlined][39m
      [5] [0m[1m_zeros[22m
    [90m    @ [39m[90m~/Softwares/Julia/julia-1.7.3/share/julia/stdlib/v1.7/LinearAlgebra/src/[39m[90m[4mLinearAlgebra.jl:409[24m[39m[90m [inlined][39m
      [6] [0m[1m\[22m[0m[1m([22m[90mF[39m::[0mLinearAlgebra.QRCompactWY[90m{Float64, Matrix{Float64}}[39m, [90mB[39m::[0mMatrix[90m{Float64}[39m[0m[1m)[22m
    [90m    @ [39m[35mLinearAlgebra[39m [90m~/Softwares/Julia/julia-1.7.3/sha

#### individuals:

In [107]:
pheno_ind_subset = lpst_ind_Y[:, samples];

In [108]:
t_ind = @elapsed gemma_L_ind = run_gemma(pheno_ind_subset, geno_ind, kinship_ind_from_strains,
                                         ["A", "B"], marker_names, 
                                         "data/GEMMA_data/GN886_pheno_ind_y.txt", "data/GEMMA_data/GN886_geno_ind.txt", "data/GEMMA_data/kinship_ind.txt", "results_ind",
                                         "/home/zyu20/Softwares/gemma-0.98.5-linux-static-AMD64")

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 248
## number of analyzed individuals = 248
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Start Eigen-Decomposition...
pve estimate =0.320037
se(pve) =0.08769


**** WARNING: Matrix G has 199 eigenvalues close to zero


================================================== 100%


**** INFO: Done.


GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 248
## number of analyzed individuals = 248
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Start Eigen-Decomposition...
pve estimate =0.0830114
se(pve) =0.0474742


...

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 248
## number of analyzed individuals = 248
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Start Eigen-Decomposition...
pve estimate =0.0220402
se(pve) =0.0339893


**** WARNING: Matrix G has 199 eigenvalues close to zero


================================================== 100%


**** INFO: Done.
Excessive output truncated after 524319 bytes.

1975.83072939

In [111]:
t_ind

1975.83072939

In [110]:
findall(isnan.(gemma_L_ind))

CartesianIndex{2}[]

In [132]:
findall(isinf.(gemma_L_ind))

5-element Vector{CartesianIndex{2}}:
 CartesianIndex(2977, 873)
 CartesianIndex(2978, 873)
 CartesianIndex(2979, 873)
 CartesianIndex(2980, 873)
 CartesianIndex(2981, 873)

In [125]:
gemma_L_ind

7321×1000 Matrix{Float64}:
 0.13297     0.00237256  0.114716    …  0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716    …  0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716    …  0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.0399133   5.74931e-5  0.176414       0.323577     0.000715

In [136]:
pwd()

"/home/zyu20/git/BulkLMM_Analyses/BXDLongevity"

In [142]:
# writedlm("output/gemma_L_strains.txt", gemma_L_strains, '\t');
# writedlm("output/gemma_L_ind.txt", gemma_L_ind, '\t');
# writedlm("output/samples1000.txt", samples, '\t');

In [134]:
abs_diff = abs.(bulkscan_results_ind_null.L[:, samples] .- gemma_L_ind);

In [143]:
mean_abs_diff = 0;

In [144]:
mean_abs_diff

0